# 内置LCEL函数用于可运行项

## Contents
* .bind()
* .assign()

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

## LCEL Chain

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [7]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

* “管道”操作符 `|` 是 LCEL 链的主要元素。
* LCEL 链中元素的顺序（从左到右）很重要。
* LCEL 链是可运行序列。

In [8]:
chain = prompt | model | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

'One curious fact about Cristiano Ronaldo is that he has his own museum dedicated to his career and achievements in his hometown of Funchal, Madeira, Portugal. The museum showcases memorabilia, trophies, and other items related to his successful career in football.'

* 链中的所有组件都是可运行的。
* 当我们调用 chain.invoke() 时，我们以有序的方式使用链中的所有组件：
    * 首先，我们对提示应用 .invoke()。
    * 然后，使用前面的输出，我们对模型应用 .invoke()。
    * 最后，使用前面的输出，我们对输出解析器应用 .invoke()。

## 使用 .bind() 在 LCEL 链中向 Runnable 添加参数
* 例如，我们可以添加一个参数，以便在模型响应达到“Ronaldo”这个词时停止响应：

In [9]:
chain = prompt | model.bind(stop=["Ronaldo"]) | output_parser

In [11]:
chain.invoke({"soccer_player": "Ronaldo"})

'One curious fact about '

## 使用 .bind() 在 LCEL 链中调用 OpenAI 函数

In [12]:
functions = [
    {
      "name": "soccerfacts",
      "description": "Curious facts about a soccer player",
      "parameters": {
        "type": "object",
        "properties": {
          "question": {
            "type": "string",
            "description": "The question for the curious facts about a soccer player"
          },
          "answer": {
            "type": "string",
            "description": "The answer to the question"
          }
        },
        "required": ["question", "answer"]
      }
    }
  ]

In [13]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "soccerfacts"}, functions= functions)
    | JsonOutputFunctionsParser()
)

In [14]:
chain.invoke(input={"soccer_player": "Mbappe"})

{'question': 'What is a curious fact about Kylian Mbappe?',
 'answer': 'Kylian Mbappe became the youngest French player to score in a World Cup at the age of 19 during the 2018 FIFA World Cup.'}

**注意：** OpenAI API 已弃用功能，转而使用工具。有关更多信息，请参见 [此处](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/openai_functions_agent/)。

## 使用.bind()来连接OpenAI工具

**注意：** 在 OpenAI Chat API 中，函数现在被视为遗留选项，已被工具所取代。如果您正在使用 OpenAI LLM 模型创建代理，您应该使用 OpenAI 工具，而不是 OpenAI 函数。

虽然通常情况下您应该使用 .bind_tools() 方法来调用工具模型，但如果您想要更低级别的控制，也可以直接绑定特定于供应商的参数：

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo").bind(tools=tools)
model.invoke("What's the weather in SF, NYC and LA?")

## assign() 函数允许将键添加到链中
* 示例：我们将创建一个名为 "operation_b" 的键，分配给一个带有 RunnableLambda 的自定义函数。
* 我们将从一个非常基本的链开始，仅包含 RunnablePassthrough：

In [20]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

chain = RunnableParallel({"original_input": RunnablePassthrough()})

In [17]:
chain.invoke("whatever")

{'original_input': 'whatever'}

* 正如您所看到的，现在这个链只将用户输入分配给“original_input”变量。
* 现在，让我们用 assign 函数添加新键“uppercase”。
* 在新的“uppercase”键中，我们将使用一个名为 `make_uppercase` 的可运行 Lambda 函数。

In [18]:
def make_uppercase(arg):
    return arg["original_input"].upper()

In [21]:
chain = RunnableParallel({"original_input": RunnablePassthrough()}).assign(uppercase=RunnableLambda(make_uppercase))

In [22]:
chain.invoke("whatever")

{'original_input': 'whatever', 'uppercase': 'WHATEVER'}

* 如你所见，链的输出现在有两个键：original_input 和 uppercase。
* 在 uppercase 键中，我们可以看到 `make_uppercase` 函数已经应用于用户输入。